# Testing functions and extracting features from objectmapping.py:


In [1]:
# %% Cell 1
import warnings
warnings.simplefilter('ignore', FutureWarning)
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from objectmapping import ObjectMapping
from mrcnn_classes import class_names
from itertools import permutations, combinations
from collections import defaultdict
import os

Using TensorFlow backend.


In [2]:
%%time
# %% Cell 2
# define the test configuration
class TestConfig(Config):
    NAME = "test"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 80

# define the model
rcnn = MaskRCNN(mode='inference', model_dir='./', config=TestConfig())

# load coco model weights
rcnn.load_weights('../data/mask_rcnn_coco.h5', by_name=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead
CPU times: user 10.4 s, sys: 629 ms, total: 11 s
Wall time: 10.9 s


In [7]:
# %% Cell 3
def area_from_box(h1, w1, h2, w2):
        return abs(h1-h2)*abs(w1-w2)
    
def depth_response(a, b):
    while True:
        print('\n')
        print(f"Are objects {a} and {b} at the same depth level?")        
        print("0: no")
        print("1: yes")
        response = input()
        if response == '1':
            break
        elif response == '0':
            break
    return int(response)

def frontbehind_response(a, b):
    while True:
        print('\n')
        print(f"If touching, is object {a} in front of or behind {b}?")        
        print("0: behind")
        print("1: in front")
        print("2: neither or not touching")
        response = input()
        if response == '0':
            break
        elif response == '1':
            break
        elif response == '2':
            break
    return int(response)

def invson_response(a, b):
    while True:
        print('\n')
        print(f"Is object {a} in object {b}?")        
        print("0: no")
        print("1: yes")
        response = input()
        if response == '0':
            break
        elif response == '1':
            break
    return int(response)

def relation_response(k, v):
    while True:
        print('\n')
        print(f"Verify '{k}' relationship for {v}")        
        print("0: False")
        print("1: True")
        response = input()
        if response == '0':
            break
        elif response == '1':
            break
    return int(response)

def depth_relation_response(k, v):
    while True:
        print('\n')
        print(f"Considering depth: Verify '{k}' relationship for {v}")        
        print("0: False")
        print("1: True")
        response = input()
        if response == '0':
            break
        elif response == '1':
            break
    return int(response)

def location_response(x):
    while True:
        print('\n')
        print(f"Verify {x}")        
        print("0: Not Correct")
        print("1: Correct")
        response = input()
        if response == '0':
            break
        elif response == '1':
            break
    return int(response)


### Prepare

In [ ]:
# %% Cell 4
with open('session.txt', 'r') as f:
    image_list = f.read().splitlines()
image_list = [i.strip('\"') for i in image_list]

<a id='input'></a>
### Start loading of next image from here by executing the following pop command:

In [ ]:
# %% Cell 5
image_filename = image_list.pop(0)

#### If the image has an error during processing, uncomment the below cell and run. Then execute the above cell to pop the next image file. Don't forget to re-comment out the cell below after running the cell. This just puts the filename to the end of the list.

In [ ]:
# %% Cell 6
# image_list.append(image_filename)

#### If you need to stop before finishing the processing of the list, you can write the remaining files in the list back out to a text file by uncommenting and running the next cell. Then when you're ready to resume, execute Cells 1-4, but make sure that the file being opened in Cell 4, is the same one written to in Cell 7.

In [ ]:
# %% Cell 7
# with open('session.txt', 'w') as f:
#     for i in image_list:
#         f.write(f"{i}\n")

In [8]:
# %% Cell 8
# load image
imagefile = image_filename
img = load_img(imagefile)
img = load_img(imagefile)
img_array = img_to_array(img)
# make prediction
results = rcnn.detect([img_array], verbose=False)
# get dictionary for first prediction
r = results[0]

# instantiate object
imap = ObjectMapping(imagefile, r, class_names)

In [ ]:
# %% Cell 9
if imap.total_objects <= 1:
    print('Not enough objects detected.')        
else:
    max_objects = 6
    image_data = defaultdict(list)
    object_bb = {}
    object_mb = {}
    areas_bb = {}
    areas_mb = {}
    object_pixels = {}
    ids = []
    
    for i in range(1, imap.total_objects+1):
        object_bb[i] = imap.get_box(i)
        object_mb[i] = imap.mass_box(i)
        object_pixels[i] = imap.mask_pixel_count(i)
        ids.append(i)
        areas_bb[i] = area_from_box(*object_bb[i])
        areas_mb[i] = area_from_box(*object_mb[i])
    
    ids = [i for i in ids if i < max_objects] # Limits the objects detected.
    image_area = imap.img_height*imap.img_width
    ptot_bb_area = sum(areas_bb.values())
    ptot_mb_area = sum(areas_mb.values())
    ptot_mask_pixels = sum(object_pixels.values())
    
    display_instances(img_array, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
    object_combos = combinations(ids, r=2) # choose combinations or permutations
    for a, b in object_combos:
        # Overall image data
        image_data['filename'].append(imap.filename)
        image_data['image height'].append(imap.img_height)
        image_data['image width'].append(imap.img_width)
        image_data['total objects'].append(imap.total_objects)
        image_data['% total pixels'].append(ptot_mask_pixels/image_area)
        image_data['% total bb area'].append(ptot_bb_area/image_area)
        image_data['% total mb area'].append(ptot_mb_area/image_area)
        
        # Object A relationship to image
        image_data['objectA'].append(imap.object_class(a))
        image_data['A % bb area'].append(areas_bb[a]/image_area)
        image_data['A % mb area'].append(areas_mb[a]/image_area)
        image_data['A diff bbmb image area'].append(image_data['A % bb area'][-1]-\
                                                  image_data['A % mb area'][-1])
        image_data['A % bb area of sum bb'].append(areas_bb[a]/ptot_bb_area)
        image_data['A % mb area of sum mb'].append(areas_mb[a]/ptot_mb_area)
        image_data['A diff bbmb box area'].append(image_data['A % bb area of sum bb'][-1]-\
                                                  image_data['A % mb area of sum mb'][-1])
        image_data['A % top bb'].append(object_bb[a][0]/imap.img_height)
        image_data['A % left bb'].append(object_bb[a][1]/imap.img_width)
        image_data['A % bottom bb'].append(object_bb[a][2]/imap.img_height)
        image_data['A % right bb'].append(object_bb[a][3]/imap.img_width)
        image_data['A % top mb'].append(object_mb[a][0]/imap.img_height)
        image_data['A % left mb'].append(object_mb[a][1]/imap.img_width)
        image_data['A % bottom mb'].append(object_mb[a][2]/imap.img_height)
        image_data['A % right mb'].append(object_mb[a][3]/imap.img_width)
        image_data['A % pixels of image area'].append(object_pixels[a]/image_area)
        image_data['A % pixels of bb area'].append(object_pixels[a]/areas_bb[a])
        image_data['A % pixels of mb area'].append(object_pixels[a]/areas_mb[a])
        image_data['A % pixels of total pixels'].append(object_pixels[a]/ptot_mask_pixels)
        image_data['A % bb height'].append(abs(object_bb[a][0]-object_bb[a][2])/imap.img_height)
        image_data['A % bb width'].append(abs(object_bb[a][1]-object_bb[a][3])/imap.img_width)
        image_data['A % mb height'].append(abs(object_mb[a][0]-object_mb[a][2])/imap.img_height)
        image_data['A % mb width'].append(abs(object_mb[a][1]-object_mb[a][3])/imap.img_width)
        
        # Object B relationship to image
        image_data['objectB'].append(imap.object_class(b))
        image_data['B % bb area'].append(areas_bb[b]/image_area)
        image_data['B % mb area'].append(areas_mb[b]/image_area)
        image_data['B diff bbmb image area'].append(image_data['B % bb area'][-1]-\
                                                  image_data['B % mb area'][-1])
        image_data['B % bb area of sum bb'].append(areas_bb[b]/ptot_bb_area)
        image_data['B % mb area of sum mb'].append(areas_mb[b]/ptot_mb_area)
        image_data['B diff bbmb box area'].append(image_data['B % bb area of sum bb'][-1]-\
                                                  image_data['B % mb area of sum mb'][-1])
        image_data['B % top bb'].append(object_bb[b][0]/imap.img_height)
        image_data['B % left bb'].append(object_bb[b][1]/imap.img_width)
        image_data['B % bottom bb'].append(object_bb[b][2]/imap.img_height)
        image_data['B % right bb'].append(object_bb[b][3]/imap.img_width)
        image_data['B % top mb'].append(object_mb[b][0]/imap.img_height)
        image_data['B % left mb'].append(object_mb[b][1]/imap.img_width)
        image_data['B % bottom mb'].append(object_mb[b][2]/imap.img_height)
        image_data['B % right mb'].append(object_mb[b][3]/imap.img_width)
        image_data['B % pixels of image area'].append(object_pixels[b]/image_area)
        image_data['B % pixels of bb area'].append(object_pixels[b]/areas_bb[b])
        image_data['B % pixels of mb area'].append(object_pixels[b]/areas_mb[b])
        image_data['B % pixels of total pixels'].append(object_pixels[b]/ptot_mask_pixels)
        image_data['B % bb height'].append(abs(object_bb[b][0]-object_bb[b][2])/imap.img_height)
        image_data['B % bb width'].append(abs(object_bb[b][1]-object_bb[b][3])/imap.img_width)
        image_data['B % mb height'].append(abs(object_mb[b][0]-object_mb[b][2])/imap.img_height)
        image_data['B % mb width'].append(abs(object_mb[b][1]-object_mb[b][3])/imap.img_width)
        
        # Object A relationship to Object B
        image_data['diffAB % bb area'].append(image_data['A % bb area'][-1]-image_data['B % bb area'][-1])
        image_data['diffAB % mb area'].append(image_data['A % mb area'][-1]-image_data['B % mb area'][-1])
        image_data['diffAB diff bbmb image area'].append(image_data['A diff bbmb image area'][-1]-\
                                                  image_data['B diff bbmb image area'][-1])
        image_data['diffAB % bb area of sum bb'].append(image_data['A % bb area of sum bb'][-1]-\
                                                        image_data['B % bb area of sum bb'][-1])
        image_data['diffAB % mb area of sum mb'].append(image_data['A % mb area of sum mb'][-1]-\
                                                        image_data['B % mb area of sum mb'][-1])
        image_data['diffAB diff bbmb box area'].append(image_data['A diff bbmb box area'][-1]-\
                                                  image_data['B diff bbmb box area'][-1])
        image_data['diffAB % top bb'].append(image_data['A % top bb'][-1]-image_data['B % top bb'][-1])
        image_data['diffAB % left bb'].append(image_data['A % left bb'][-1]-image_data['B % left bb'][-1])
        image_data['diffAB % bottom bb'].append(image_data['A % bottom bb'][-1]-image_data['B % bottom bb'][-1])
        image_data['diffAB % right bb'].append(image_data['A % right bb'][-1]-image_data['B % right bb'][-1])
        image_data['diffAB % top mb'].append(image_data['A % top mb'][-1]-image_data['B % top mb'][-1])
        image_data['diffAB % left mb'].append(image_data['A % left mb'][-1]-image_data['B % left mb'][-1])
        image_data['diffAB % bottom mb'].append(image_data['A % bottom mb'][-1]-image_data['B % bottom mb'][-1])
        image_data['diffAB % right mb'].append(image_data['A % right mb'][-1]-image_data['B % right mb'][-1])
        image_data['diffAB % pixels of image area'].append(image_data['A % pixels of image area'][-1]-\
                                                           image_data['B % pixels of image area'][-1])
        image_data['diffAB % pixels of bb area'].append(image_data['A % pixels of bb area'][-1]-\
                                                       image_data['B % pixels of bb area'][-1])
        image_data['diffAB % pixels of mb area'].append(image_data['A % pixels of mb area'][-1]-\
                                                       image_data['B % pixels of mb area'][-1])
        image_data['diffAB % pixels of total pixels'].append(image_data['A % pixels of total pixels'][-1]-\
                                                            image_data['B % pixels of total pixels'][-1])
        image_data['diffAB % bb height'].append(image_data['A % bb height'][-1]-image_data['B % bb height'][-1])
        image_data['diffAB % bb width'].append(image_data['A % bb width'][-1]-image_data['B % bb width'][-1])
        image_data['diffAB % mb height'].append(image_data['A % mb height'][-1]-image_data['B % mb height'][-1])
        image_data['diffAB % mb width'].append(image_data['A % mb width'][-1]-image_data['B % mb width'][-1])
        
        atruebox = imap.create_box_mask(*object_mb[a])
        btruebox = imap.create_box_mask(*object_mb[b])
        overlapbox = np.bitwise_and(atruebox, btruebox) 
        image_data['AB % massbox overlap'].append(np.sum(overlapbox)/areas_mb[a])
        
        maska = imap.get_mask(a).copy()
        maskb = imap.get_mask(b).copy()
        toplinea = imap.object_topline(a)
        toplineb = imap.object_topline(b)
        image_data['a_on_b'].append(np.any(np.bitwise_and(maska, toplineb)))
        image_data['b_on_a'].append(np.any(np.bitwise_and(maskb, toplinea)))
        
        # Annotating Results from objectmappy.py
        imap.object_outline(a,b, show_id=True).show()
        relations = dict(*imap.object_relations(a,b).values())
        for k, v in relations.items():
            if v:
                image_data[k].append(list(v))
                image_data[f"check {k}"].append(relation_response(k,v))
                image_data[f"depth check {k}"].append(depth_relation_response(k,v))
            else:
                image_data[k].append(0)
                image_data[f"check {k}"].append(relation_response(k,v))
                image_data[f"depth check {k}"].append(depth_relation_response(k,v))
        
        location = imap.object_location(a, grid=True)
        image_data['object A location'].append(location)
        image_data['check object A location'].append(location_response(location))
        
        location = imap.object_location(b, grid=True)
        image_data['object B location'].append(location)
        image_data['check object B location'].append(location_response(location))
        
        location = imap.grid_coords(a, grid=True)
        image_data['object A grid'].append(location)
        image_data['check object A grid'].append(location_response(location))
        
        location = imap.grid_coords(b, grid=True)
        image_data['object B grid'].append(location)
        image_data['check object B grid'].append(location_response(location))

        
        # Annotation responses for feature analysis
        imap.object_outline(a,b, show_id=True).show()
        depth = depth_response(a,b)
        if depth == 1:
            image_data['same depth'].append(depth)
            print('\n')
            image_data['frontbehind'].append(frontbehind_response(a,b))
            image_data['a in b'].append(invson_response(a,b))
        else:
            image_data['same depth'].append(depth)
            image_data['frontbehind'].append(2)
            image_data['a in b'].append(0)
            

[Back to image input](#input)

In [12]:
# %% Cell 10
if len(image_data) == 107:
    df = pd.DataFrame.from_dict(image_data)
else:
    print("Double-check dictionary size. There should be 107 key:value combinations.")
df

,filename,image height,image width,total objects,% total pixels,% total bb area,% total mb area,objectA,A % bb area,A % mb area,...,check in,depth check in,object A location,check object A location,object B location,check object B location,object A grid,check object A grid,object B grid,check object B grid
0,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,elephant,0.157633,0.086263,...,1,1,"{'vertical': {'bottom', 'center'}, 'horizontal...",1,"{'vertical': {'top', 'center'}, 'horizontal': ...",1,"{(C, 3), (B, 2), (C, 2), (B, 3)}",1,"{(A, 1), (B, 1), (B, 2), (A, 2)}",1
1,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,elephant,0.157633,0.086263,...,1,1,"{'vertical': {'bottom', 'center'}, 'horizontal...",1,"{'vertical': {'bottom'}, 'horizontal': {'cente...",1,"{(C, 3), (B, 2), (C, 2), (B, 3)}",1,"{(C, 2)}",1
2,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,elephant,0.157633,0.086263,...,1,1,"{'vertical': {'bottom', 'center'}, 'horizontal...",1,"{'vertical': {'top', 'bottom', 'center'}, 'hor...",1,"{(C, 3), (B, 2), (C, 2), (B, 3)}",1,"{(A, 2), (A, 1), (B, 1), (B, 2), (C, 2), (C, 1)}",1
3,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,elephant,0.157633,0.086263,...,1,1,"{'vertical': {'bottom', 'center'}, 'horizontal...",1,"{'vertical': {'bottom'}, 'horizontal': {'right'}}",1,"{(C, 3), (B, 2), (C, 2), (B, 3)}",1,"{(C, 3)}",1
4,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.010010,0.006529,...,1,1,"{'vertical': {'top', 'center'}, 'horizontal': ...",1,"{'vertical': {'bottom'}, 'horizontal': {'cente...",1,"{(A, 1), (B, 1), (B, 2), (A, 2)}",1,"{(C, 2)}",1
5,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.010010,0.006529,...,1,1,"{'vertical': {'top', 'center'}, 'horizontal': ...",1,"{'vertical': {'top', 'bottom', 'center'}, 'hor...",1,"{(A, 1), (B, 1), (B, 2), (A, 2)}",1,"{(A, 2), (A, 1), (B, 1), (B, 2), (C, 2), (C, 1)}",1
6,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.010010,0.006529,...,1,1,"{'vertical': {'top', 'center'}, 'horizontal': ...",1,"{'vertical': {'bottom'}, 'horizontal': {'right'}}",1,"{(A, 1), (B, 1), (B, 2), (A, 2)}",1,"{(C, 3)}",1
7,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.012049,0.006872,...,1,1,"{'vertical': {'bottom'}, 'horizontal': {'cente...",1,"{'vertical': {'top', 'bottom', 'center'}, 'hor...",1,"{(C, 2)}",1,"{(A, 2), (A, 1), (B, 1), (B, 2), (C, 2), (C, 1)}",1
8,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.012049,0.006872,...,1,1,"{'vertical': {'bottom'}, 'horizontal': {'cente...",1,"{'vertical': {'bottom'}, 'horizontal': {'right'}}",1,"{(C, 2)}",1,"{(C, 3)}",1
9,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,horse,0.241970,0.132062,...,1,1,"{'vertical': {'top', 'bottom', 'center'}, 'hor...",1,"{'vertical': {'bottom'}, 'horizontal': {'right'}}",1,"{(A, 2), (A, 1), (B, 1), (B, 2), (C, 2), (C, 1)}",1,"{(C, 3)}",1


# Warning: running the following cell writes the dataframe to file. Do not run if there was a processing error or there were not enough objects detected. This will just duplicate the previous data in the file.

In [13]:
# %% Cell 11
if df.shape[1] == 107:
    df.to_csv('../data/sessions/function_testing.csv', mode='a',
              header=not os.path.isfile('../data/sessions/function_testing.csv'), index=False)
else:
    print("Double-check dataframe dimension.")

[Back to image input](#input)

In [14]:
# %% Cell 12
checkcsv = pd.read_csv('../data/sessions/function_testing.csv')
checkcsv

,filename,image height,image width,total objects,% total pixels,% total bb area,% total mb area,objectA,A % bb area,A % mb area,...,check in,depth check in,object A location,check object A location,object B location,check object B location,object A grid,check object A grid,object B grid,check object B grid
0,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,elephant,0.157633,0.086263,...,1,1,"{'vertical': {'bottom', 'center'}, 'horizontal...",1,"{'vertical': {'top', 'center'}, 'horizontal': ...",1,"{('C', 3), ('B', 2), ('C', 2), ('B', 3)}",1,"{('A', 1), ('B', 1), ('B', 2), ('A', 2)}",1
1,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,elephant,0.157633,0.086263,...,1,1,"{'vertical': {'bottom', 'center'}, 'horizontal...",1,"{'vertical': {'bottom'}, 'horizontal': {'cente...",1,"{('C', 3), ('B', 2), ('C', 2), ('B', 3)}",1,"{('C', 2)}",1
2,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,elephant,0.157633,0.086263,...,1,1,"{'vertical': {'bottom', 'center'}, 'horizontal...",1,"{'vertical': {'top', 'bottom', 'center'}, 'hor...",1,"{('C', 3), ('B', 2), ('C', 2), ('B', 3)}",1,"{('A', 2), ('A', 1), ('B', 1), ('B', 2), ('C',...",1
3,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,elephant,0.157633,0.086263,...,1,1,"{'vertical': {'bottom', 'center'}, 'horizontal...",1,"{'vertical': {'bottom'}, 'horizontal': {'right'}}",1,"{('C', 3), ('B', 2), ('C', 2), ('B', 3)}",1,"{('C', 3)}",1
4,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.010010,0.006529,...,1,1,"{'vertical': {'top', 'center'}, 'horizontal': ...",1,"{'vertical': {'bottom'}, 'horizontal': {'cente...",1,"{('A', 1), ('B', 1), ('B', 2), ('A', 2)}",1,"{('C', 2)}",1
5,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.010010,0.006529,...,1,1,"{'vertical': {'top', 'center'}, 'horizontal': ...",1,"{'vertical': {'top', 'bottom', 'center'}, 'hor...",1,"{('A', 1), ('B', 1), ('B', 2), ('A', 2)}",1,"{('A', 2), ('A', 1), ('B', 1), ('B', 2), ('C',...",1
6,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.010010,0.006529,...,1,1,"{'vertical': {'top', 'center'}, 'horizontal': ...",1,"{'vertical': {'bottom'}, 'horizontal': {'right'}}",1,"{('A', 1), ('B', 1), ('B', 2), ('A', 2)}",1,"{('C', 3)}",1
7,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.012049,0.006872,...,1,1,"{'vertical': {'bottom'}, 'horizontal': {'cente...",1,"{'vertical': {'top', 'bottom', 'center'}, 'hor...",1,"{('C', 2)}",1,"{('A', 2), ('A', 1), ('B', 1), ('B', 2), ('C',...",1
8,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,dog,0.012049,0.006872,...,1,1,"{'vertical': {'bottom'}, 'horizontal': {'cente...",1,"{'vertical': {'bottom'}, 'horizontal': {'right'}}",1,"{('C', 2)}",1,"{('C', 3)}",1
9,../images/animals_test_mixed_hidden-5.jpg,384,576,5,0.211326,0.428467,0.235957,horse,0.241970,0.132062,...,1,1,"{'vertical': {'top', 'bottom', 'center'}, 'hor...",1,"{'vertical': {'bottom'}, 'horizontal': {'right'}}",1,"{('A', 2), ('A', 1), ('B', 1), ('B', 2), ('C',...",1,"{('C', 3)}",1


[Back to image input](#input)